In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
path = "/data/pcpe_01.csv"
df = pd.read_csv(path, delimiter=';')

In [6]:
columns_num = [
    'DIA_LANCAMENTO', 
    'MES_LANCAMENTO',
    'VALOR_TRANSACAO',
    'VALOR_SALDO',
]

columns_cat = [
    'TIPO',
    'CNAB',
    'NATUREZA_SALDO'
]

target = ['I-a']
y = df[target].values
y = np.nan_to_num(y)

In [27]:
from mlex.features.columns import CompositeTranformer

tranformer = CompositeTranformer(
    numeric_features=columns_num,
    categorical_features=columns_cat
)

Xt = tranformer.fit_transform(df)

print(Xt)

[[<bound method TransformerMixin.fit_transform of NumericalTransfomer()>
  <bound method TransformerMixin.fit_transform of CategoricalOneHotTransfomer()>]]


In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

encoder = OneHotEncoder()
scaler = MinMaxScaler()

X_onehot = encoder.fit_transform(df[columns_cat]).toarray()
X_num = scaler.fit_transform(df[columns_num])

Xt = np.column_stack([X_onehot, X_num])

Xt

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 7.50000000e-08, 3.70279998e-05],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.00000000e-07, 3.71987144e-05],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.00000000e-08, 3.06547964e-04],
       ...,
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.81818182e-01, 1.19500000e-06, 4.91828803e-05],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.81818182e-01, 0.00000000e+00, 8.02358685e-07],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.81818182e-01, 0.00000000e+00, 8.02358685e-07]])

In [40]:
from mlex.utils.splits import PastFutureSplit

split = PastFutureSplit()

X_train, X_test, y_train, y_test = split.train_test_split(Xt,y)

In [41]:
from mlex.features.sequences import SequenceTransfomer
p = SequenceTransfomer()
np.asarray(X_train)
np.asarray(y_train)
data_train = p.transform( 
    X = X_train,
    y = y_train
)

In [66]:
import keras
import keras.optimizers.adam

In [76]:
from mlex.pipelines.simple_pipeline import SimplePipeline
from mlex.models.models import SimpleRNNModel

my_model = SimpleRNNModel(
                    input_shape=Xt.shape
                    ).get_model
 

model = model_simplernn = SimplePipeline(
    numeric_features=columns_num,
    categorical_features=columns_cat,
    final_model= my_model
)